### --- [Day 7: Handy Haversacks](https://adventofcode.com/2020/day/7) ---

You land at the regional airport in time for your next flight. In fact, it looks like you'll even have time to grab some food: all flights are currently delayed due to issues in luggage processing.

Due to recent aviation regulations, many rules (your puzzle input) are being enforced about bags and their contents; bags must be color-coded and must contain specific quantities of other color-coded bags. Apparently, nobody responsible for these regulations considered how long they would take to enforce!

For example, consider the following rules:

light red bags contain 1 bright white bag, 2 muted yellow bags.
dark orange bags contain 3 bright white bags, 4 muted yellow bags.
bright white bags contain 1 shiny gold bag.
muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.
shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.
dark olive bags contain 3 faded blue bags, 4 dotted black bags.
vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.
faded blue bags contain no other bags.
dotted black bags contain no other bags.

These rules specify the required contents for 9 bag types. In this example, every faded blue bag is empty, every vibrant plum bag contains 11 bags (5 faded blue and 6 dotted black), and so on.

You have a shiny gold bag. If you wanted to carry it in at least one other bag, how many different bag colors would be valid for the outermost bag? (In other words: how many colors can, eventually, contain at least one shiny gold bag?)

In the above rules, the following options would be available to you:

    A bright white bag, which can hold your shiny gold bag directly.
    A muted yellow bag, which can hold your shiny gold bag directly, plus some other bags.
    A dark orange bag, which can hold bright white and muted yellow bags, either of which could then hold your shiny gold bag.
    A light red bag, which can hold bright white and muted yellow bags, either of which could then hold your shiny gold bag.

So, in this example, the number of bag colors that can eventually contain at least one shiny gold bag is 4.

How many bag colors can eventually contain at least one shiny gold bag? (The list of rules is quite long; make sure you get all of it.)


In [1]:
import re

INPUT_FILE = 'input_d7.txt'

class Graph:
    '''
    Graph representation(s) of all bags from rule input.
    Each edge represents a bag that can contain 1+ bags of the
    vertex bag's color.
    '''
    
    def __init__(self, input_file):
        self.contains = {}
        self.contained_by = {}
        self.read_file(input_file)
            
    def add_edge(self, outer, bag, qty):
        '''
        Adds an edge to the graph. Each vertex is a bag color, and its edges
        connect to the bag(s) that contain(s) it.
        :param outer: string representing the outer bag's color
        :param bag: string representing the inner bag's color
        :param qty: the quantity of the inner bag that the outer bag may hold
        '''
        if bag not in self.contained_by:
            # Create the vertex
            self.contained_by[bag] = []
        if outer not in self.contains:
            self.contains[outer] = []
            
        # Add the contained_by graph edge, which is a tuple of (outer, quantity)
        self.contained_by[bag].append((outer, int(qty)))
        
        # Add the contains graph edge
        self.contains[outer].append((bag, int(qty))) 
        

    def find_all_containing(self, bag):
        '''
        Solution to Part 1 - BFS the graph for all vertices that can
        contain the specified bag
        :param bag: the inner bag to search for
        '''
        result = set()
        queue = [bag]
        
        while len(queue) > 0:
            vertex = queue.pop(0)
            if vertex in self.contained_by:
                # If there are contained-by edges for this vertex, add all to queue
                for edge in self.contained_by[vertex]:
                    queue.append(edge[0])

            # Don't add the original vertex to the result
            if len(result) > 0 or vertex != bag:
                result.add(vertex)
        return result
    
    def count_contained(self, bag):
        '''
        Solution to part 2 - recursive count of all contained bags
        :param bag: the color of the containing bag to count
        :return: the total number of bags contained by the specified bag
        '''
        # Base case: bag contains no children
        if bag not in self.contains:
            return 0
        else:
            # Contained count is qty of child + (qty of child * count_contained(child)) for all children
            tot = 0
            for child in self.contains[bag]:
                tot += child[1]
                tot += child[1] * self.count_contained(child[0])
            return tot
        
    def read_row(self, row):
        '''
        Takes a text row from the input and extracts the relevant data, adding
        edges to the graph as encountered
        :param row: string data containing rule
        '''
        outer_pat = r"(\w+ \w+) bags contain \d+"  # Match the outer bag only when it contains 1+ bags
        inner_pat = r"(\d+) (\w+ \w+) bags?[,.]?"  # Match any inner bag
        
        outer_match = re.match(outer_pat, row)
        if outer_match:
            inner_matches = re.finditer(inner_pat, row)
            for match in inner_matches:
                # match[1] is the quantity, match[2] is the inner bag color
                self.add_edge(outer_match[1], match[2], match[1])
        
    def read_file(self, input_file):
        '''
        Reads the input file and extracts all rules
        :param input_file:
        '''
        with open(input_file) as f:
            for row in f.readlines():
                self.read_row(row)

In [2]:
# Example input
bg_example = Graph('input2_d7.txt')
bg_example.contained_by

{'bright white': [('light red', 1), ('dark orange', 3)],
 'muted yellow': [('light red', 2), ('dark orange', 4)],
 'shiny gold': [('bright white', 1), ('muted yellow', 2)],
 'faded blue': [('muted yellow', 9), ('dark olive', 3), ('vibrant plum', 5)],
 'dark olive': [('shiny gold', 1)],
 'vibrant plum': [('shiny gold', 2)],
 'dotted black': [('dark olive', 4), ('vibrant plum', 6)]}

In [3]:
print(bg_example.find_all_containing('shiny gold'))
assert len(bg_example.find_all_containing('shiny gold')) == 4

{'dark orange', 'muted yellow', 'bright white', 'light red'}


In [4]:
# Part 1 solution **********************************************
bg = Graph(INPUT_FILE)
result = bg.find_all_containing('shiny gold')
print('Part 1 solution:', len(result), result)


Part 1 solution: 151 {'vibrant green', 'dotted black', 'vibrant black', 'posh tan', 'striped coral', 'dull blue', 'dotted magenta', 'vibrant teal', 'dim teal', 'drab fuchsia', 'bright indigo', 'faded lavender', 'clear aqua', 'dotted gray', 'muted tan', 'plaid white', 'mirrored purple', 'pale crimson', 'dark lavender', 'dark cyan', 'dotted fuchsia', 'dull white', 'dull red', 'drab lime', 'bright salmon', 'faded green', 'wavy lime', 'mirrored turquoise', 'clear teal', 'posh teal', 'dim blue', 'dull gold', 'dotted cyan', 'bright red', 'pale green', 'posh white', 'bright chartreuse', 'muted cyan', 'muted lavender', 'dull purple', 'bright green', 'mirrored green', 'shiny cyan', 'dull green', 'mirrored lavender', 'muted gray', 'pale lime', 'posh violet', 'shiny black', 'dark beige', 'light coral', 'mirrored salmon', 'posh silver', 'muted coral', 'plaid coral', 'shiny turquoise', 'posh indigo', 'dull plum', 'posh crimson', 'shiny silver', 'clear fuchsia', 'drab beige', 'plaid gray', 'pale aqu

#### Part 2
It's getting pretty expensive to fly these days - not because of ticket prices, but because of the ridiculous number of bags you need to buy!

Consider again your shiny gold bag and the rules from the above example:

    faded blue bags contain 0 other bags.
    dotted black bags contain 0 other bags.
    vibrant plum bags contain 11 other bags: 5 faded blue bags and 6 dotted black bags.
    dark olive bags contain 7 other bags: 3 faded blue bags and 4 dotted black bags.

So, a single shiny gold bag must contain 1 dark olive bag (and the 7 bags within it) plus 2 vibrant plum bags (and the 11 bags within each of those): 1 + 1*7 + 2 + 2*11 = 32 bags!

Of course, the actual rules have a small chance of going several levels deeper than this example; be sure to count all of the bags, even if the nesting becomes topologically impractical!

Here's another example:

shiny gold bags contain 2 dark red bags.
dark red bags contain 2 dark orange bags.
dark orange bags contain 2 dark yellow bags.
dark yellow bags contain 2 dark green bags.
dark green bags contain 2 dark blue bags.
dark blue bags contain 2 dark violet bags.
dark violet bags contain no other bags.

In this example, a single shiny gold bag must contain 126 other bags.

How many individual bags are required inside your single shiny gold bag?

In [5]:
# Part 2 example
bg2 = Graph('input3_d7.txt')
bg2.contains

{'shiny gold': [('dark red', 2)],
 'dark red': [('dark orange', 2)],
 'dark orange': [('dark yellow', 2)],
 'dark yellow': [('dark green', 2)],
 'dark green': [('dark blue', 2)],
 'dark blue': [('dark violet', 2)]}

In [6]:
assert bg2.count_contained('shiny gold') == 126

In [7]:
# Part 2 solution
bg2 = Graph(INPUT_FILE)
bg2.count_contained('shiny gold')

41559